In [1]:
import os,sys
from pathlib import Path
currentdir = Path(os.getcwd())
sys.path.insert(0, os.path.dirname(currentdir.parent))
sys.path.insert(0, os.path.dirname(currentdir))
print(str(sys.path[0:3]))

['c:\\Users\\anaxa\\Documents\\Projects\\BAMT', 'c:\\Users\\anaxa\\Documents\\Projects', 'c:\\Users\\anaxa\\Documents\\Projects\\BAMT\\tutorials']


In [2]:
import BAMT.Networks as Nets
import BAMT.Preprocessors as pp

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from pgmpy.estimators import K2Score

C:\Users\anaxa\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\anaxa\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [3]:
data = pd.read_csv(r'../Data/vk_data.csv')
data

,sex,age,city_id,has_high_education,relation,num_of_relatives,followers_count,status,mobile_phone,twitter,...,"car, cars, sale, tires","sale, clothes, size, price, sewing","ingredients, cooking, dishes, salt, spices","theater, performance, actors","photographer, photography, show, wedding, photographer",top1_interes,top2_interes,top3_interes,top4_interes,top5_interes
0,1,50.0,2,0,0,2,27,0,0,0,...,0.201622,3.397819e-02,0.003674,3.081348e-01,1.070007e-02,"theater, performance, actors","car, cars, sale, tires","participation, festival, exhibition","client, project, internet, business, company","home, renovation, decoration, installation"
1,2,36.0,2,0,0,2,6,0,0,0,...,0.060985,2.529792e-02,0.020964,1.153278e-02,3.497006e-03,"book, reading, language, word, literature","music, concerts, rock, tickets, festivals","school, education, question, topic","client, project, internet, business, company","war, history, army, politics"
2,1,65.0,2,0,0,2,25,0,0,0,...,0.000009,2.007164e-09,0.100547,1.566668e-03,4.916993e-02,"client, project, internet, business, company","skin, face, care, cosmitology","ingredients, cooking, dishes, salt, spices","treatment, doctors, tests, clinic, health","photographer, photography, show, wedding, phot..."
3,2,30.0,2,0,0,2,52,0,1,0,...,0.143803,5.324164e-03,0.027627,1.481971e-02,1.903577e-02,"war, history, army, politics","car, cars, sale, tires","film, series, cinema, genre","music, concerts, rock, tickets, festivals","football, match, russia, team, goalkeeper"
4,2,36.0,2,0,0,2,18,0,0,0,...,0.037360,4.527913e-02,0.087596,4.388889e-02,7.207429e-02,"home, renovation, decoration, installation","ingredients, cooking, dishes, salt, spices","game, team, competition, sport","massage, meditation, yoga","shop, price, gifts, internet"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2,50.0,2,0,0,2,24,0,1,0,...,0.021852,1.240507e-02,0.000604,2.096222e-02,6.373252e-03,"music, concerts, rock, tickets, festivals","skin, face, care, cosmitology","film, series, cinema, genre","handmade, materials, products","client, project, internet, business, company"
29996,1,37.0,2,0,0,2,0,4,0,0,...,0.010729,2.396505e-02,0.085240,3.546863e-02,3.498729e-02,"participation, festival, exhibition","ingredients, cooking, dishes, salt, spices","shop, price, gifts, internet","beauty, hair, manicure, salon","game, team, competition, sport"
29997,2,55.0,2,0,4,60,4,0,1,0,...,0.000000,0.000000e+00,0.001092,8.027095e-09,3.005514e-10,"music, concerts, rock, tickets, festivals","color, flowers, interior, design","book, reading, language, word, literature","ingredients, cooking, dishes, salt, spices","game, team, competition, sport"
29998,1,39.0,2,0,0,2,102,0,0,0,...,0.013108,2.857535e-02,0.127433,7.420912e-02,3.401194e-02,"ingredients, cooking, dishes, salt, spices","vacation, sea, accommodation, cost","home, renovation, decoration, installation","theater, performance, actors","game, team, competition, sport"


In [4]:
cols = ['age', 'sex', 'has_pets', 'is_parent', 'relation', 'is_driver', 'tr_per_month', 'median_tr', 'mean_tr']
data = data[cols]

In [5]:
encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

p = pp.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])
discretized_data, est = p.apply(data)

In [6]:
bn = Nets.HybridBN(has_logit=True, use_mixture=True) # init BN
info = p.info
info

{'types': {'age': 'cont',
  'sex': 'disc_num',
  'has_pets': 'disc_num',
  'is_parent': 'disc_num',
  'relation': 'disc_num',
  'is_driver': 'disc_num',
  'tr_per_month': 'cont',
  'median_tr': 'cont',
  'mean_tr': 'cont'},
 'signs': {'age': 'pos',
  'tr_per_month': 'pos',
  'median_tr': 'pos',
  'mean_tr': 'pos'}}

In [7]:
bn.add_nodes(info)
bn.add_edges(discretized_data,  scoring_function=('K2',K2Score))

  0%|          | 19/1000000 [00:01<17:21:13, 16.01it/s]


In [8]:
bn.get_info()

,name,node_type,data_type,parents,parents_types
0,age,ConditionalMixtureGaussian,cont,"[sex, is_parent, relation]","[disc_num, disc_num, disc_num]"
1,sex,Discrete,disc_num,"[is_parent, relation, is_driver]","[disc_num, disc_num, disc_num]"
2,has_pets,ConditionalLogit (LogisticRegression),disc_num,"[tr_per_month, sex]","[cont, disc_num]"
3,is_parent,Discrete,disc_num,[relation],[disc_num]
4,relation,Discrete,disc_num,[],[]
5,is_driver,Discrete,disc_num,[],[]
6,tr_per_month,ConditionalMixtureGaussian,cont,"[median_tr, mean_tr, sex, is_parent]","[cont, cont, disc_num, disc_num]"
7,median_tr,ConditionalMixtureGaussian,cont,"[age, sex, is_parent]","[cont, disc_num, disc_num]"
8,mean_tr,ConditionalMixtureGaussian,cont,"[age, median_tr, is_parent]","[cont, cont, disc_num]"


In [9]:
bn.fit_parameters(data)

In [11]:
test = data[['sex', 'has_pets', 'is_parent', 'relation', 'is_driver', 'tr_per_month', 'median_tr', 'mean_tr']].sample(100)
test.reset_index(inplace=True, drop=True)
test

,sex,has_pets,is_parent,relation,is_driver,tr_per_month,median_tr,mean_tr
0,1,0,1,4,0,2.258385,1678.775,2268.355455
1,2,0,0,6,1,35.010291,262.880,566.448108
2,2,0,1,0,0,8.592723,2295.450,7184.371024
3,1,0,0,4,0,30.721332,271.500,601.088148
4,1,0,1,4,0,36.438512,528.000,1363.052183
...,...,...,...,...,...,...,...,...
95,1,0,0,0,0,16.423128,300.000,1123.356108
96,2,0,0,0,0,35.875439,466.575,1024.765505
97,2,0,0,0,0,8.300966,400.000,310.000000
98,1,0,0,0,0,5.764560,931.520,4805.202000


In [12]:
bn.predict(test, 11)

100%|██████████| 100/100 [00:37<00:00,  2.69it/s]


{'age': []}